In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import test as tp
import sys

np.set_printoptions(threshold=sys.maxsize)  # to print total matrix


In [2]:
A = 3 * 10 ** -4
E = 70 * 10 ** 9
node = []
node.append(tp.Point(0, 3, "01"))
node.append(tp.Point(3, 3, "02"))
node.append(tp.Point(6, 3, "03"))
node.append(tp.Point(3, 0, "04"))
node.append(tp.Point(6, 0, "05"))
node.append(tp.Point(0, 0, "06"))

element = []
element.append(tp.Element(node[2-1], node[1-1], A=A, E=E))
element.append(tp.Element(node[2-1], node[3-1], A=A, E=E))
element.append(tp.Element(node[1-1], node[6-1], A=A, E=E))
element.append(tp.Element(node[3-1], node[5-1], A=A, E=E))
element.append(tp.Element(node[2-1], node[4-1], A=A, E=E))
element.append(tp.Element(node[2-1], node[6-1], A=A, E=E))
element.append(tp.Element(node[1-1], node[4-1], A=A, E=E))
element.append(tp.Element(node[2-1], node[5-1], A=A, E=E))
element.append(tp.Element(node[3-1], node[4-1], A=A, E=E))
element.append(tp.Element(node[4-1], node[5-1], A=A, E=E))
element.append(tp.Element(node[4-1], node[6-1], A=A, E=E))

element[1-1].make_element_K_with_help(node[1], node[0])
element[2-1].make_element_K_with_help(node[2-1], node[3-1])
element[3-1].make_element_K_with_help(node[1-1], node[6-1])
element[4-1].make_element_K_with_help(node[3-1], node[5-1])
element[5-1].make_element_K_with_help(node[2-1], node[4-1])
element[6-1].make_element_K_with_help(node[2-1], node[6-1])
element[7-1].make_element_K_with_help(node[1-1], node[4-1])
element[8-1].make_element_K_with_help(node[2-1], node[5-1])
element[9-1].make_element_K_with_help(node[3-1], node[4-1])
element[10-1].make_element_K_with_help(node[4-1], node[5-1])
element[11-1].make_element_K_with_help(node[4-1], node[6-1])


In [17]:
K = np.zeros(12 * 12)  # n*n total
K_help = tp.make_K_help(12)  # n total
# print(K_help)
# print(element[0].K_help)
# print(len(element))
for i in range(len(element)):
    K = element[i].merge_K(K, K_help)
print(K.reshape((12,12)))

[[ 9474873.73415292 -2474873.73415292 -7000000.                0.
         0.                0.         -2474873.73415292  2474873.73415292
         0.                0.                0.                0.        ]
 [-2474873.73415292  9474873.73415292        0.                0.
         0.                0.          2474873.73415292 -2474873.73415292
         0.                0.                0.         -7000000.        ]
 [-7000000.                0.         18949747.46830584        0.
  -7000000.                0.                0.                0.
  -2474873.73415292  2474873.73415292 -2474873.73415292 -2474873.73415292]
 [       0.                0.                0.         11949747.46830584
         0.                0.                0.         -7000000.
   2474873.73415292 -2474873.73415292 -2474873.73415292 -2474873.73415292]
 [       0.                0.         -7000000.                0.
   9474873.73415292  2474873.73415292 -2474873.73415292 -2474873.73415292
        

In [18]:
K_modified = np.array([])
new_K_help = tp.make_K_help(8, 1, 1)  # show where is the modified K
for p, v in np.ndenumerate(K_help):
    if v in new_K_help:
        K_modified = np.append(K_modified, K[p])

K_modified = K_modified.reshape([8, 8])  # shape the modified.
print(str(K_modified))

[[ 9474873.73415292 -2474873.73415292 -7000000.                0.
         0.                0.         -2474873.73415292  2474873.73415292]
 [-2474873.73415292  9474873.73415292        0.                0.
         0.                0.          2474873.73415292 -2474873.73415292]
 [-7000000.                0.         18949747.46830584        0.
  -7000000.                0.                0.                0.        ]
 [       0.                0.                0.         11949747.46830584
         0.                0.                0.         -7000000.        ]
 [       0.                0.         -7000000.                0.
   9474873.73415292  2474873.73415292 -2474873.73415292 -2474873.73415292]
 [       0.                0.                0.                0.
   2474873.73415292  9474873.73415292 -2474873.73415292 -2474873.73415292]
 [-2474873.73415292  2474873.73415292        0.                0.
  -2474873.73415292 -2474873.73415292 18949747.46830584        0.        ]
 [ 24

In [19]:
K_modified_inv = np.linalg.inv(K_modified)
F = np.zeros(8)
F[1] = -50000
F[3] = -100000
F[5] = -50000
F_show = F.reshape((8, 1))
displacements = np.dot(K_modified_inv, F_show)
displacements_show = displacements
print(displacements_show)

[[ 1.35242734e-03]
 [-8.49528449e-03]
 [ 5.23825481e-19]
 [-1.63778087e-02]
 [-1.35242734e-03]
 [-8.49528449e-03]
 [-1.44614784e-19]
 [-1.36729540e-02]]


In [9]:
displacements_total = np.zeros(12)  # total dimension, rows or columns
iteration = -1
temp = 0
new2_K_help = new_K_help.copy()
new2_K_help = new2_K_help[
              0:-1:8]  # n(the last number in parentheses) is the number of dimension in matrix, the dimension of new_K_help
# print(len(K_help))
for i in range(len(new2_K_help)):
    new2_K_help[i] = new2_K_help[i][0:2]
for p, v in np.ndenumerate(K_help):
    if temp == v[0:2]:
        continue
    temp = v[0:2]
    if v[0:2] in new2_K_help:
        iteration += 1
        displacements_total[(p[0] // 12)] = displacements[
            iteration]  # n is the dimension of main matrix... total K dimension
K = K.reshape([12, 12])  # n*n total
displacements_total = displacements_total.reshape((12, 1))
F_total = np.dot(K, displacements_total)
# print(F_total)
print(displacements_total)

[[ 1.35242734e-03]
 [ 0.00000000e+00]
 [-8.49528449e-03]
 [ 0.00000000e+00]
 [ 5.23825481e-19]
 [ 0.00000000e+00]
 [-1.63778087e-02]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]]
